<a href="https://colab.research.google.com/github/gombergere/arthomas/blob/master/PipeXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://towardsdatascience.com/https-medium-com-tirthajyoti-build-pipelines-with-pandas-using-pdpipe-cade6128cd31
#Art et maniere de traiter un fichier de donneees Csv
#...c'est un passage obligé du DL, pas passionnant, mais faut faire avec.

In [16]:
import pandas as pd
import numpy as np
!pip install pdpipe

In [17]:
import pdpipe as pdp

/usr/local/lib/python3.6/dist-packages/pdpipe/__init__.py:48: UserWarning: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pdpipe/__init__.py", line 42, in <module>
    from . import sklearn_stages
  File "/usr/local/lib/python3.6/dist-packages/pdpipe/sklearn_stages.py", line 6, in <module>
    from skutil.preprocessing import scaler_by_params
ModuleNotFoundError: No module named 'skutil'

  warnings.warn(tb)
/usr/local/lib/python3.6/dist-packages/pdpipe/__init__.py:50: UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
  "pdpipe: Scikit-learn or skutil import failed. Scikit-learn"


In [6]:
from google.colab import files
file=files.upload()

Saving USA_Housing.csv to USA_Housing.csv


In [0]:
#df sera le symbole simplifié du fichier
df = pd.read_csv("USA_Housing.csv")

In [37]:
#le chiffre 2 représente le nb de décimales
round(df.sample(3),2)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size
2124,65627.05,7.97,6.86,2.21,44808.30,1614667.53,71899 Cole Throughway Apt. 835\nNorth Wendymou...,Big
2860,69876.36,5.00,7.25,6.13,20038.86,839629.51,Unit 7240 Box 7803\nDPO AA 77851-4397,Big
1645,71639.30,6.57,5.61,3.44,28995.09,945931.19,"81945 Adams Cliffs Suite 280\nNew Mason, DE 40217",Medium


In [9]:
df.shape

(5000, 7)

In [10]:
df.columns

Index(['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population', 'Price', 'Address'],
      dtype='object')

In [11]:
round(df.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
Avg. Area Income,5000.0,68583.11,10657.99,17796.63,61480.56,68804.29,75783.34,107701.75
Avg. Area House Age,5000.0,5.98,0.99,2.64,5.32,5.97,6.65,9.52
Avg. Area Number of Rooms,5000.0,6.99,1.01,3.24,6.30,7.00,7.67,10.76
Avg. Area Number of Bedrooms,5000.0,3.98,1.23,2.00,3.14,4.05,4.49,6.50
Area Population,5000.0,36163.52,9925.65,172.61,29403.93,36199.41,42861.29,69621.71
Price,5000.0,1232072.65,353117.63,15938.66,997577.14,1232669.38,1471210.20,2469065.59


In [0]:
# créer un code pour une grandeur
def size(n):
    if n<=4:
        return 'Small'
    elif 4<n<=6:
        return 'Medium'
    else:
        return 'Big'

df['House_size']=df['Avg. Area Number of Rooms'].apply(size)

In [0]:
df['House_size']=df['Avg. Area Number of Rooms'].apply(size)

In [38]:
round(df.sample(3),2)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size
135,74411.06,5.53,7.21,6.44,43537.50,1393746.76,"4930 James Flats\nNew Shannon, NJ 75829-8040",Big
1732,54858.82,5.96,5.66,4.42,36228.84,850114.60,"411 Janet Island\nSamuelmouth, IL 41084-3705",Medium
2697,60891.73,6.59,6.32,3.19,41217.12,1045348.23,"0574 David Row\nSouth Jacquelineshire, ME 7549...",Big


In [0]:
drop_age = pdp.ColDrop('Avg. Area House Age')

In [0]:
df2 = drop_age(df)

In [39]:
round(df2.sample(3))

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size
4372,85207.0,7.0,5.0,40946.0,2024121.0,"0562 Sullivan Isle\nEast Edwardside, IA 49505",Big
96,66264.0,7.0,4.0,49830.0,1441422.0,"001 Steve Plaza\nJessicastad, UT 25190",Big
199,59363.0,8.0,3.0,44723.0,1442945.0,"8581 Johnson Ranch Apt. 905\nJacquelinefurt, M...",Big


In [40]:
drop_address=pdp.ColDrop('Address')
df21=drop_address(df2)
round(df21.sample(3))

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,House_size
2268,89511.0,8.0,6.0,29766.0,1580727.0,Big
1937,66644.0,6.0,3.0,36859.0,1450770.0,Medium
850,63596.0,7.0,3.0,19345.0,864703.0,Big


In [21]:
pipeline = pdp.ColDrop('Avg. Area House Age')
pipeline+= pdp.OneHotEncode('House_size')
df3 = pipeline(df)
round(df3.sample(3))

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
612,84090.0,7.0,2.0,25128.0,1409033.0,"503 Edward Bypass Suite 138\nChambersborough, ...",0,0
1223,62559.0,8.0,5.0,44522.0,1029037.0,Unit 2965 Box 1848\nDPO AE 20627,0,0
3739,57847.0,8.0,4.0,40209.0,1111360.0,"5048 Marquez Meadows Suite 774\nJessicamouth, ...",0,0
2780,78734.0,6.0,3.0,36688.0,1850514.0,"8600 Bailey Garden\nNew Debra, IL 09574",0,0
1243,56281.0,8.0,6.0,29142.0,962502.0,"8263 Shaw Ridges Suite 890\nSouth Evanland, AZ...",0,0


In [22]:
  def price_tag(x):
    if x>250000:
        return 'keep'
    else:
        return 'drop'
pipeline = pdp.ColDrop('Avg. Area House Age')
pipeline+= pdp.OneHotEncode('House_size')
pipeline+=pdp.ApplyByCols('Price',price_tag,'Price_tag',drop=False)
df4 = pipeline(df)
df4.shape

(5000, 9)

In [41]:
round(df4.sample(3),2)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Price_tag,Address,House_size_Medium,House_size_Small
1662,76388.55,6.85,2.20,36841.18,1114729.72,keep,"PSC 1483, Box 6662\nAPO AP 78821-4369",0,0
289,57782.42,6.33,2.46,33426.85,861321.58,keep,Unit 2107 Box 1980\nDPO AA 76733,0,0
4604,56631.90,5.59,2.44,51351.72,1374117.07,keep,"884 Jordan Streets Apt. 271\nDawsontown, OR 98005",1,0


In [42]:
pipeline = pdp.ColDrop('Avg. Area House Age')
pipeline+= pdp.OneHotEncode('House_size')
pipeline+=pdp.ApplyByCols('Price',price_tag,'Price_tag',drop=False)
pipeline+=pdp.ValDrop(['drop'],'Price_tag')
pipeline+= pdp.ColDrop('Price_tag')
df5 = pipeline(df)
df5.shape

(4990, 8)

In [0]:
#Scikit-learn scaling
pipeline_scale = pdp.Scale('StandardScaler',exclude_columns=['House_size_Medium','House_size_Small'])
df6 = pipeline_scale(df5)
round(df6.sample(5),3)

In [0]:
pipeline_tokenize=pdp.TokenizeWords('Address')

In [0]:
df7 = pipeline_tokenize(df5)

In [46]:
df7.sample(3)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
1983,68570.082280,6.406176,4.09,44476.819292,1.024788e+06,"[Unit, 8667, Box, 6237, DPO, AE, 76811-0261]",0,0
3069,35454.714659,6.018647,4.50,59636.402553,1.077806e+06,"[Unit, 4700, Box, 1880, DPO, AP, 18074]",0,0
3346,73197.707659,6.165414,3.16,22829.493270,1.161233e+06,"[USCGC, Nichols, FPO, AE, 98812]",0,0


In [0]:
def extract_state(token):
    return str(token[-2])

In [0]:
pipeline_state = pdp.ApplyByCols('Address',extract_state,result_columns='State')

In [0]:
df8=pipeline_state(df7)

In [51]:
round(df8.sample(3),3)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,State,House_size_Medium,House_size_Small
4604,56631.895,5.590,2.44,51351.721,1374117.067,OR,1,0
3468,67855.245,8.304,5.22,34836.603,1222576.820,MO,0,0
764,77587.947,6.820,4.37,37280.442,1485145.661,SD,0,0
